### Import libraries

In [1]:
import datetime
from datetime import timezone
import json
import logging
import os
import sys
import credentials
from getpass import getpass

import readchar
import requests
from garth.exc import GarthHTTPError

from garminconnect import (
    Garmin,
    GarminConnectAuthenticationError,
    GarminConnectConnectionError,
    GarminConnectTooManyRequestsError,
)

### Retrieving login credentials (read from `credentials.py` or use saved login in cache)

In [2]:
# Configure debug logging
# logging.basicConfig(level=logging.DEBUG)
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load environment variables if defined
email = credentials.email
password = credentials.password
tokenstore = os.getenv("GARMINTOKENS") or "~/.garminconnect"
tokenstore_base64 = os.getenv("GARMINTOKENS_BASE64") or "~/.garminconnect_base64"

### Define utility functions

In [ ]:
# Define options to be displayed in the menu
menu_options = {
    "1": f"Get activity data",
    "2": f"Get activity data (compatible with garminconnect-ha)",
    "3": f"Get daily steps data",
    "4": f"Get heart rate data",
    "5": f"Get blood pressure data of onw week",
    "6": f"Get floors data",
    "7": f"Get sleep data",
    "8": f"Get stress data",
    "9": f"Get respiration data",
    "0": f"Get SpO2 data",
    "a": f"Get Heart Rate Variability data (HRV)",
    "b": f"Get all day stress data",
    "c": f"Get daily steps data of one week",
    "q": "Exit",
}

api = None

format = '%d-%m-%Y'

def get_credentials():
    """Get user credentials."""

    email = input("Login e-mail: ")
    password = getpass("Enter password: ")

    return email, password

def print_menu():
    """Print examples menu."""
    for key in menu_options.keys():
        print(f"{key} -- {menu_options[key]}")

def switch(api, i):
    """Run selected API call."""

    # Exit example program
    if i == "q":
        print("Be active, generate some data to fetch next time ;-) Bye!")
        sys.exit()

    # Skip requests if login failed
    if api:
        try:
            print(f"\n\nExecuting: {menu_options[i]}\n")

            # USER STATISTIC SUMMARIES
            if i == "1":
                # Get activity data for 'YYYY-MM-DD'
                save_json(
                    f"api.get_stats('{startdate.isoformat()}')",
                    api.get_stats(startdate.isoformat()),
                )
            elif i == "2":
                # Get activity data (to be compatible with garminconnect-ha)
                save_json(
                    f"api.get_user_summary('{startdate.isoformat()}')",
                    api.get_user_summary(startdate.isoformat()),
                )

            # USER STATISTICS LOGGED
            elif i == "3":
                # Get steps data for 'YYYY-MM-DD'
                save_json(
                    f"api.get_steps_data('{startdate.isoformat()}')",
                    api.get_steps_data(startdate.isoformat()),
                )

            elif i == "4":
                # Get heart rate data for 'YYYY-MM-DD'
                save_json(
                    f"api.get_heart_rates('{startdate.isoformat()}')",
                    api.get_heart_rates(startdate.isoformat()),
                )

            elif i == "5":
                # Get daily blood pressure data for 'YYYY-MM-DD' to 'YYYY-MM-DD'
                save_json(
                    f"api.get_blood_pressure('{startdate.isoformat()}, {enddate.isoformat()}')",
                    api.get_blood_pressure(startdate.isoformat(), enddate.isoformat()),
                )

            elif i == "6":
                # Get daily floors data for 'YYYY-MM-DD'
                save_json(
                    f"api.get_floors('{startdate.isoformat()}')",
                    api.get_floors(startdate.isoformat()),
                )

            elif i == "7":
                # Get sleep data for 'YYYY-MM-DD'
                save_json(
                    f"api.get_sleep_data('{startdate.isoformat()}')",
                    api.get_sleep_data(startdate.isoformat()),
                )
            elif i == "8":
                # Get stress data for 'YYYY-MM-DD'
                save_json(
                    f"api.get_stress_data('{startdate.isoformat()}')",
                    api.get_stress_data(startdate.isoformat()),
                )
            elif i == "9":
                # Get respiration data for 'YYYY-MM-DD'
                save_json(
                    f"api.get_respiration_data('{startdate.isoformat()}')",
                    api.get_respiration_data(startdate.isoformat()),
                )
            elif i == "0":
                # Get SpO2 data for 'YYYY-MM-DD'
                save_json(
                    f"api.get_spo2_data('{startdate.isoformat()}')",
                    api.get_spo2_data(startdate.isoformat()),
                )

            elif i == "a":
                # Get Heart Rate Variability (hrv) data
                save_json(
                    f"api.get_hrv_data('{startdate.isoformat()}')",
                    api.get_hrv_data(startdate.isoformat()),
                )

            elif i == "b":
                # Get all day stress data for date
                save_json(
                    f"api.get_all_day_stress('{startdate.isoformat()}')",
                    api.get_all_day_stress(startdate.isoformat()),
                )

            elif i == "c":
                # Get daily step data for 'YYYY-MM-DD'
                display_json(
                    f"api.get_daily_steps('{startdate.isoformat()}, {enddate.isoformat()}')",
                    api.get_daily_steps(startdate.isoformat(), enddate.isoformat()),
                )


        except (
            GarminConnectConnectionError,
            GarminConnectAuthenticationError,
            GarminConnectTooManyRequestsError,
            requests.exceptions.HTTPError,
            GarthHTTPError,
        ) as err:
            logger.error(err)
        except KeyError:
            # Invalid menu option chosen
            pass
    else:
        print("Could not login to Garmin Connect, try again later.")

# Define function to save retrieved jsons from GarminConnect
def save_json(api_call, output):
    """Save API output to file."""

    dashed = "-" * 20
    header = f"{dashed} {api_call} {dashed}"
    footer = "-" * len(header)

    print(header)

    if isinstance(output, (int, str, dict, list)):
        with open(f"{api_call}.json", "w") as file:
            file.write(json.dumps(output, indent=4))
    else:
        print(output)

    print(footer)

def init_api(email, password):
    """Initialize Garmin API with your credentials."""

    try:
        # Using Oauth1 and OAuth2 token files from directory
        print(
            f"Trying to login to Garmin Connect using token data from directory '{tokenstore}'...\n"
        )

        # Using Oauth1 and Oauth2 tokens from base64 encoded string
        # print(
        #     f"Trying to login to Garmin Connect using token data from file '{tokenstore_base64}'...\n"
        # )
        # dir_path = os.path.expanduser(tokenstore_base64)
        # with open(dir_path, "r") as token_file:
        #     tokenstore = token_file.read()

        garmin = Garmin()
        garmin.login(tokenstore)

    except (FileNotFoundError, GarthHTTPError, GarminConnectAuthenticationError):
        # Session is expired. You'll need to log in again
        print(
            "Login tokens not present, login with your Garmin Connect credentials to generate them.\n"
            f"They will be stored in '{tokenstore}' for future use.\n"
        )
        try:
            # Ask for credentials if not set as environment variables
            if not email or not password:
                email, password = get_credentials()

            garmin = Garmin(
                email=email, password=password, is_cn=False, prompt_mfa=get_mfa
            )
            garmin.login()
            # Save Oauth1 and Oauth2 token files to directory for next login
            garmin.garth.dump(tokenstore)
            print(
                f"Oauth tokens stored in '{tokenstore}' directory for future use. (first method)\n"
            )
            # Encode Oauth1 and Oauth2 tokens to base64 string and safe to file for next login (alternative way)
            token_base64 = garmin.garth.dumps()
            dir_path = os.path.expanduser(tokenstore_base64)
            with open(dir_path, "w") as token_file:
                token_file.write(token_base64)
            print(
                f"Oauth tokens encoded as base64 string and saved to '{dir_path}' file for future use. (second method)\n"
            )
        except (
            FileNotFoundError,
            GarthHTTPError,
            GarminConnectAuthenticationError,
            requests.exceptions.HTTPError,
        ) as err:
            logger.error(err)
            return None

    return garmin


def get_mfa():
    """Get MFA."""

    return input("MFA one-time code: ")





### Loop to keep API connection open and allow to ask for more data

In [10]:
# Initialize variable to keep track if date that user want to get data from is set
valid_start_date = False
#valid_end_date = False

# Main program loop
while True:
    # Display header and login
    print("\n*** Garmin Connect API ***\n")

    # Init API
    if not api:
        api = init_api(email, password)

    if api:
        while not valid_start_date:
            # Get start_date that user want to get data from
            try:
                startdate = datetime.datetime.strptime(input("Enter the date to get data from (DD-MM-YYYY): "), format).date()
                #startdate = datetime.datetime.strptime("07-02-2025", format)
                enddate = startdate - datetime.timedelta(days=7)
                valid_start_date = True
            except ValueError as e:
                print("Invalid date format, please try again.")
                continue
        '''        
        while not valid_end_date:
            # Get end_date that user want to get data from
            try:
                #enddate = datetime.datetime.strptime(input("Enter the end date to get data from (DD-MM-YYYY): "), format)
                enddate = datetime.datetime.strptime("09-02-2025", format)
                valid_end_date = True
            except ValueError as e:
                print("Invalid date format, please try again.")
                continue
        '''
        # Display menu
        print_menu()
        option = input(print("Make your selection: "))
        print(option)
        switch(api, option)
    else:
        api = init_api(email, password)


*** Garmin Connect API ***

1 -- Get activity data
2 -- Get activity data (compatible with garminconnect-ha)
3 -- Get daily steps data
4 -- Get heart rate data
5 -- Get blood pressure data of onw week
6 -- Get floors data
7 -- Get sleep data
8 -- Get stress data
9 -- Get respiration data
0 -- Get SpO2 data
a -- Get Heart Rate Variability data (HRV)
b -- Get all day stress data
c -- Get daily steps data of one week
q -- Exit
Make your selection: 
3


Executing: Get daily steps data

-------------------- api.get_steps_data('2025-02-07') --------------------
--------------------------------------------------------------------------

*** Garmin Connect API ***

1 -- Get activity data
2 -- Get activity data (compatible with garminconnect-ha)
3 -- Get daily steps data
4 -- Get heart rate data
5 -- Get blood pressure data of onw week
6 -- Get floors data
7 -- Get sleep data
8 -- Get stress data
9 -- Get respiration data
0 -- Get SpO2 data
a -- Get Heart Rate Variability data (HRV)
b -- Get a

SystemExit: 

c:\Users\tomas\Desktop\PhD\Garmin\python-garminconnect\.conda\Lib\site-packages\IPython\core\interactiveshell.py:3557: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
